In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset, random_split
import pandas as pd
import numpy as np
import os
import random
from sklearn.metrics import r2_score
from copy import deepcopy

## ハイパーパラメータの設定

In [2]:
cfg = {
    "batch_size": 8,
    "input_dim": 5,
    "hidden_dims": [32, 16],
    "output_dim": 1,
    "activation": "Tanh",
    "learning_rate": 1e-3,
    "num_epochs": 100,
    "dropout": 0.1,
    "shuffle": True,
    "val_rate": 0.3,
    "num_epochs": 100,
    "patience": 5,
    "save_path": r"C:\Users\ryoya\MasterThesis\MT_Furuie\results\Miwa_FFNN\Trial_2\best_model_trial_2_3_12.pth",
    "metrics_excel_path": r"C:\Users\ryoya\MasterThesis\MT_Furuie\results\Miwa_FFNN\Trial_2\metrics_trial_2_3_12.xlsx"
}

## データの前処理

In [3]:
# データのパス指定
data_folder = r"C:\Users\ryoya\MasterThesis\MT_Furuie\data\Miwa_FFNN_Data\Trial_1"
# data_folder = r"C:\Users\RYOYA\OneDrive\ドキュメント\データ整理\美和（修論）\Miwa_FFNN\Trial_1"
data_file_name = r"Miwa_data_for_FFNN.xlsx"
idx_file_name = r"Miwa_flood_idx_for_FFNN.xlsx"


data_path = os.path.join(data_folder, data_file_name)
idx_path = os.path.join(data_folder, idx_file_name)


# input, output変数の列番号を指定（0始まり）
# タイムラグもここで指定
input_cols = [2, 2, 1, 1, 26]
input_lags = [0, 1, 1, 2, 0]
output_cols = [1]
output_lags = [0]

# ファイルの読み込み
d_all = pd.read_excel(data_path, header=0)
idx_list = pd.read_excel(idx_path, header=0)
col_trial = 10 # 【要変更】どの列がtrain, testを指定する列か

train_idx = idx_list[idx_list.iloc[:, col_trial] == 'train']
test_idx = idx_list[idx_list.iloc[:, col_trial] == 'test']


In [4]:
# 必要なデータの取り出し（train）
x_train = []
y_train = []

# inputの取り出し
for i in range(train_idx.shape[0]):
    s = int(train_idx.iloc[i, 0]) - 1
    e = int(train_idx.iloc[i, 1]) - 1

    # ---- X（入力データ）: 各列をその列ごとの lag 分だけずらして抽出
    cols_block = []
    for col, lag in zip(input_cols, input_lags):
        start = s - lag
        end = e - lag
        # lag分だけ前の行を取り出す（指定範囲のまま、NaN埋め不要）
        x_part = d_all.iloc[start:end+1, col].to_numpy().reshape(-1, 1)
        cols_block.append(x_part)

    X_seg = np.hstack(cols_block)  # (L, len(input_cols))

    # 区間ごとに格納
    x_train.append(X_seg)

# ---- すべての区間を縦方向に結合
x_train = np.vstack(x_train)


# outputの取り出し
for i in range(train_idx.shape[0]):
    s = int(train_idx.iloc[i, 0]) - 1
    e = int(train_idx.iloc[i, 1]) - 1

    # ---- X（入力データ）: 各列をその列ごとの lag 分だけずらして抽出
    cols_block = []
    for col, lag in zip(output_cols, output_lags):
        start = s - lag
        end = e - lag
        # lag分だけ前の行を取り出す（指定範囲のまま、NaN埋め不要）
        y_part = d_all.iloc[start:end+1, col].to_numpy().reshape(-1, 1)
        cols_block.append(y_part)

    Y_seg = np.hstack(cols_block)  # (L, len(input_cols))

    # 区間ごとに格納
    y_train.append(Y_seg)

# ---- すべての区間を縦方向に結合
y_train = np.vstack(y_train)

In [5]:
# データの標準化関数
def standardize_by_column(X):
    """列ごとに標準化し、平均と標準偏差を返す"""
    mean = np.mean(X, axis=0)
    std  = np.std(X, axis=0, ddof=0)

    # 標準偏差が0の列は0除算を防ぐ
    std[std == 0] = 1.0

    X_std = (X - mean) / std
    return X_std, mean, std


In [6]:
# データの標準化
x_train_std, x_mean, x_std = standardize_by_column(x_train)
y_train_std, y_mean, y_std = standardize_by_column(y_train)

x_train_params = pd.DataFrame({'mean': x_mean, 'std': x_std})
y_train_params = pd.DataFrame({'mean': y_mean, 'std': y_std})

print(x_train_params)
print(y_train_params)


          mean          std
0    96.992753    73.664817
1    96.641515    73.883039
2  1130.457090  1725.251261
3  1127.694644  1726.527855
4    39.008712    42.079069
          mean          std
0  1132.640482  1724.254377


## 初期値の定義関数

In [7]:
# シード値のセット関数

def set_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    

In [8]:
# 重みの初期値の定義

def init_weights(model, activation="ReLU"):
    """
    活性化関数に応じて初期値を設定する関数
    ReLU → He初期化、 Sigmoid/Tanh → Xavier初期化
    """
    for m in model.modules():
        if isinstance(m, nn.Linear):
            if activation in ["ReLU", "LeakyReLU", "ELU"]:
                nn.init.kaiming_uniform_(m.weight, nonlinearity='relu')
            elif activation in ["Sigmoid", "Tanh"]:
                nn.init.xavier_uniform_(m.weight)
            else:
                # その他の活性化関数用（デフォルト）
                nn.init.xavier_uniform_(m.weight)
            if m.bias is not None:
                nn.init.zeros_(m.bias)

## モデル構造の定義

In [9]:
# モデルの定義
def build_ffnn(cfg: dict) -> nn.Module:
    """
    Feed Forward Neural Network (FFNN) を構築する関数。
    cfg（辞書）でネットワーク構造・活性化関数・ドロップアウト率などを指定。

    Parameters
    ----------
    cfg : dict
        モデル設定を含む辞書。例：
        {
            "input_dim": 10,
            "hidden_dims": [256, 128, 64],
            "output_dim": 1,
            "activation": "ReLU",
            "dropout": 0.2
        }

    Returns
    -------
    model : torch.nn.Module
        指定条件に基づくFFNNモデル
    """

    # --- 活性化関数マッピング ---
    activation_funcs = {
        "ReLU": nn.ReLU(),
        "LeakyReLU": nn.LeakyReLU(),
        "ELU": nn.ELU(),
        "Sigmoid": nn.Sigmoid(),
        "Tanh": nn.Tanh(),
        "GELU": nn.GELU()
    }

    act = activation_funcs.get(cfg.get("activation", "ReLU"), nn.ReLU())
    dropout_rate = cfg.get("dropout", 0.0)
    hidden_dims = cfg.get("hidden_dims", [])
    input_dim = cfg["input_dim"]
    output_dim = cfg["output_dim"]

    layers = []
    in_dim = input_dim

    # --- 隠れ層を順に構築 ---
    for h in hidden_dims:
        layers.append(nn.Linear(in_dim, h))
        layers.append(act)
        if dropout_rate > 0:
            layers.append(nn.Dropout(dropout_rate))
        in_dim = h

    # --- 出力層（活性化関数なし） ---
    layers.append(nn.Linear(in_dim, output_dim))

    model = nn.Sequential(*layers)
    return model
    

## 学習ループ関数の定義

In [10]:

def train_with_early_stopping(
    model,
    train_loader,
    val_loader,
    criterion,
    optimizer,
    num_epochs=100,
    patience=5,
    device=None,            # ← デフォルトは None に
    verbose=True,
    min_delta=0.0,
    save_path="best_model.pth",
    meta: dict | None = None,  # ← 保存したい付随情報（cfg, x_mean 等）をまとめて渡す
):
    """
    Early Stopping 付き学習ループ（回帰: MSE想定）。
    - 検証損失が最良のときの state_dict を保存
    - 学習後にベスト重みを復元
    - meta に渡された dict は torch.save の payload に同梱
    """
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    best_val_loss = float("inf")
    best_state = None
    wait = 0
    history = {"train_loss": [], "val_loss": []}

    for epoch in range(1, num_epochs + 1):
        # ----- Train -----
        model.train()
        running_train, n_train = 0.0, 0

        for xb, yb in train_loader:
            xb = xb.to(device)
            yb = yb.to(device)

            optimizer.zero_grad(set_to_none=True)
            pred = model(xb)
            loss = criterion(pred, yb)
            loss.backward()
            optimizer.step()

            bs = xb.size(0)
            running_train += loss.item() * bs
            n_train += bs

        train_loss = running_train / max(n_train, 1)

        # ----- Validate -----
        model.eval()
        running_val, n_val = 0.0, 0
        with torch.no_grad():
            for xb, yb in val_loader:
                xb = xb.to(device)
                yb = yb.to(device)
                pred = model(xb)
                loss = criterion(pred, yb)

                bs = xb.size(0)
                running_val += loss.item() * bs
                n_val += bs

        val_loss = running_val / max(n_val, 1)

        history["train_loss"].append(train_loss)
        history["val_loss"].append(val_loss)

        if verbose:
            print(f"Epoch {epoch:03d} | train {train_loss:.6f} | val {val_loss:.6f}")

        # ----- Early Stopping 判定 -----
        if (best_val_loss - val_loss) > min_delta:
            best_val_loss = val_loss
            best_state = deepcopy(model.state_dict())

            payload = {"model_state_dict": best_state}
            if meta is not None:
                payload.update(meta)  # 例: {"cfg": cfg, "x_mean": ..., ...}
            torch.save(payload, save_path)

            wait = 0
        else:
            wait += 1
            if wait >= patience:
                if verbose:
                    print(f"Early stopping at epoch {epoch} (best val {best_val_loss:.6f})")
                break

    # ベスト重みを復元
    if best_state is not None:
        model.load_state_dict(best_state)
    else:
        # 念のため保存済みファイルからの復元も試す
        try:
            ckpt = torch.load(save_path, map_location=device)
            model.load_state_dict(ckpt["model_state_dict"])
        except Exception:
            pass

    return model, history


## 評価関数の定義

In [11]:
# 評価関数

def evaluate_regression(y_true, y_pred):
    """
    RMSE と 決定係数 R² を計算する関数
    -----------------------------------
    Parameters
    ----------
    y_true : array-like or torch.Tensor
        正解値
    y_pred : array-like or torch.Tensor
        予測値
    
    Returns
    -------
    metrics : dict
        {"RMSE": ..., "R2": ...}
    """

    # Tensor の場合は NumPy 配列に変換
    if isinstance(y_true, torch.Tensor):
        y_true = y_true.detach().cpu().numpy()
    if isinstance(y_pred, torch.Tensor):
        y_pred = y_pred.detach().cpu().numpy()

    # 平均二乗誤差 (MSE)
    mse = np.mean((y_true - y_pred) ** 2)
    rmse = np.sqrt(mse)

    # 決定係数 R²
    r2 = r2_score(y_true, y_pred)

    return {"RMSE": rmse, "R2": r2}

In [12]:
def eval_on_loader_original_scale(model, loader, device, y_mean=None, y_std=None, y_scaler=None):
    model.eval()
    preds_std, trues_std = [], []

    with torch.no_grad():
        for xb, yb in loader:
            xb = xb.to(device)
            yb = yb.to(device)
            out = model(xb)  # 標準化後スケールでの予測
            preds_std.append(out.cpu())
            trues_std.append(yb.cpu())

    y_pred_std = torch.cat(preds_std, dim=0).numpy()
    y_true_std = torch.cat(trues_std, dim=0).numpy()

    # ---- 元スケールへ逆変換 ----
    if y_scaler is not None:
        # scikit-learnのStandardScalerを使っている場合
        y_pred = y_scaler.inverse_transform(y_pred_std)
        y_true = y_scaler.inverse_transform(y_true_std)
    else:
        # 手元の平均・標準偏差で逆変換する場合
        y_mean = np.asarray(y_mean).reshape(1, -1)
        y_std  = np.asarray(y_std).reshape(1, -1)
        y_pred = y_pred_std * y_std + y_mean
        y_true = y_true_std * y_std + y_mean

    return evaluate_regression(y_true, y_pred)

## シード値ごとのループ関数

In [13]:
# 1 回のシード実行で Dataset 作成 → split → DataLoader → モデル初期化 → 学習（early stopping）→ 評価（元スケール）

def run_one_seed(
    seed: int,
    cfg: dict,
    x_train_std: np.ndarray,
    y_train_std: np.ndarray,
    x_mean: np.ndarray,
    x_std: np.ndarray,
    y_mean: np.ndarray,
    y_std: np.ndarray,
):
    """
    1つのseedについて、学習→評価（train/val）を実行してメトリクスを返す。
    戻り値: metrics_train(dict), metrics_val(dict), model, history(dict)
    """
    set_seed(seed)
    g = torch.Generator().manual_seed(seed)

    # ---- データ準備（テンソル化）
    # y を (N, 1) にそろえる（必要な場合のみ）
    y_arr = y_train_std if y_train_std.ndim == 2 else y_train_std[:, None]
    X = torch.from_numpy(np.ascontiguousarray(x_train_std)).float()
    Y = torch.from_numpy(np.ascontiguousarray(y_arr)).float()

    assert X.shape[0] == Y.shape[0], "x と y のサンプル数が一致しません"

    full_ds = TensorDataset(X, Y)

    # ---- split（seedごとに再現性を持たせる）
    N = len(full_ds)
    n_val = int(cfg.get("val_rate", 0.2) * N)
    n_train = N - n_val
    train_ds, val_ds = random_split(full_ds, [n_train, n_val], generator=g)

    # ---- DataLoader
    batch_size = cfg.get("batch_size", 128)
    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True,
                              num_workers=0, pin_memory=True, generator=g)
    val_loader   = DataLoader(val_ds,   batch_size=batch_size, shuffle=False,
                              num_workers=0, pin_memory=True)

    # ---- モデル & 初期化
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = build_ffnn(cfg).to(device)
    init_weights(model, activation=cfg.get("activation", "ReLU"))

    # ---- 損失 & Optimizer
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(),
                           lr=cfg.get("learning_rate", 1e-3),
                           weight_decay=cfg.get("weight_decay", 0.0))

    # ---- 学習（early stopping）
    save_path = cfg.get("save_path", f"best_model_seed{seed}.pth")
    meta = {"cfg": cfg, "x_mean": x_mean, "x_std": x_std, "y_mean": y_mean, "y_std": y_std}
    model, history = train_with_early_stopping(
        model=model,
        train_loader=train_loader,
        val_loader=val_loader,
        criterion=criterion,
        optimizer=optimizer,
        num_epochs=cfg.get("num_epochs", 100),
        patience=cfg.get("patience", 5),
        device=device,
        verbose=cfg.get("verbose", True),
        min_delta=cfg.get("min_delta", 1e-5),
        save_path=save_path,
        meta=meta,
    )

    # ---- 評価（標準化前スケールへ戻す）
    metrics_train = eval_on_loader_original_scale(
        model, train_loader, device, y_mean=y_mean, y_std=y_std
    )
    metrics_val = eval_on_loader_original_scale(
        model, val_loader, device, y_mean=y_mean, y_std=y_std
    )

    return metrics_train, metrics_val, model, history


## 学習

In [14]:
seeds = [0, 1, 2, 3, 4]
metrics_train_list = []
metrics_val_list = []


for s in seeds:
    mt, mv, model_s, hist_s = run_one_seed(
        seed=s,
        cfg=cfg,
        x_train_std=x_train_std,
        y_train_std=y_train_std,
        x_mean = x_mean,
        x_std = x_std,
        y_mean=y_mean,
        y_std=y_std,
    )
    print(f"[seed {s}] train={mt}, val={mv}")
    metrics_train_list.append(mt)
    metrics_val_list.append(mv)

    print(f"seed値{s}での学習が終了しました")


Epoch 001 | train 0.623513 | val 0.154779
Epoch 002 | train 0.228129 | val 0.107470
Epoch 003 | train 0.213173 | val 0.102869
Epoch 004 | train 0.161070 | val 0.085944
Epoch 005 | train 0.140684 | val 0.061588
Epoch 006 | train 0.158255 | val 0.057099
Epoch 007 | train 0.128544 | val 0.053397
Epoch 008 | train 0.111204 | val 0.053300
Epoch 009 | train 0.108616 | val 0.047374
Epoch 010 | train 0.126336 | val 0.046444
Epoch 011 | train 0.137475 | val 0.058911
Epoch 012 | train 0.127553 | val 0.045373
Epoch 013 | train 0.100552 | val 0.044031
Epoch 014 | train 0.110752 | val 0.043805
Epoch 015 | train 0.097790 | val 0.049492
Epoch 016 | train 0.095830 | val 0.045038
Epoch 017 | train 0.105212 | val 0.042840
Epoch 018 | train 0.104582 | val 0.047734
Epoch 019 | train 0.110949 | val 0.046288
Epoch 020 | train 0.099620 | val 0.043149
Epoch 021 | train 0.109662 | val 0.044616
Epoch 022 | train 0.108634 | val 0.050545
Early stopping at epoch 22 (best val 0.042840)
[seed 0] train={'RMSE': 459.1

## 結果の保存

In [15]:
# ---- 行列（seed × 指標）を作る
train_mat = np.array([[m['RMSE'], m['R2']] for m in metrics_train_list], dtype=float)
val_mat   = np.array([[m['RMSE'], m['R2']] for m in metrics_val_list],   dtype=float)

# ---- サマリーDF（seedごとのRMSE/R2）
df = pd.DataFrame({
    "seed": seeds,
    "train_RMSE": train_mat[:, 0],
    "train_R2":   train_mat[:, 1],
    "val_RMSE":   val_mat[:, 0],
    "val_R2":     val_mat[:, 1],
})

# ---- 平均行を追加
mean_row = {
    "seed": "mean",
    "train_RMSE": train_mat[:, 0].mean(),
    "train_R2":   train_mat[:, 1].mean(),
    "val_RMSE":   val_mat[:, 0].mean(),
    "val_R2":     val_mat[:, 1].mean(),
}
df = pd.concat([df, pd.DataFrame([mean_row])], ignore_index=True)

# ---- Excel に保存（summary + 生行列）
out_xlsx = cfg.get("metrics_excel_path", "metrics_by_seed.xlsx")
os.makedirs(os.path.dirname(out_xlsx) or ".", exist_ok=True)

with pd.ExcelWriter(out_xlsx) as w:
    df.to_excel(w, sheet_name="summary", index=False)
    pd.DataFrame(train_mat, columns=["RMSE", "R2"]).to_excel(
        w, sheet_name="train_matrix", index_label="seed_idx"
    )
    pd.DataFrame(val_mat, columns=["RMSE", "R2"]).to_excel(
        w, sheet_name="val_matrix", index_label="seed_idx"
    )

print("✅ 保存完了:", out_xlsx)
print(df)

✅ 保存完了: C:\Users\ryoya\MasterThesis\MT_Furuie\results\Miwa_FFNN\Trial_2\metrics_trial_2_3_12.xlsx
   seed  train_RMSE  train_R2    val_RMSE    val_R2
0     0  459.135852  0.932268  356.885163  0.951873
1     1  441.199220  0.929597  432.508809  0.945872
2     2  469.043953  0.933308  314.071029  0.955187
3     3  486.814049  0.923661  327.315282  0.959733
4     4  329.052085  0.957031  628.551777  0.901537
5  mean  437.049032  0.935173  411.866412  0.942840
